# plot_amsr_lwp.ipynb

Code to plot maps and distributions of LWP from AMSR2 satellite and processed model output.

In [ ]:
import xarray as xr
import pandas as pn
import numpy as np
import numpy.ma as ma
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import capricorn_figure_props as fprops
import capricorn_functions as cfn
import capricorn_constants as const
from simulations import simulations
from glob import glob
from importlib import reload
reload(cfn)
reload(const)


## Load AMSR2 measurements from file

In [ ]:
# load AMSR data in time order (using np.sort on filenames)
amsr_files = np.sort(glob('data/amsr2/RSS_*'))
amsr_lwp = []
for file in amsr_files:
    print(f'[INFO] loading {file}', flush=True)
    da = xr.open_dataset(file)['cloud_liquid_water']
    amsr_lwp.append(da)

# flatten amsr list into 10x 2D dataarrays
amsr_lwp = [da.isel({'pass':i}) for da in amsr_lwp for i in range(2)]
amsr_lwp = amsr_lwp[1:-1]


## Load model output
Pre-processed:
* compute-domain-lwp.py (to calculate 3D LWP from 4D cloud liquid water mass and air density)
* regrid-to-amsr.py (to interpolate model output onto AMSR spatial grid).

In [ ]:
fill_val = -99999  # defined in regrid-to-amsr.py
files = glob('data/model/*liquid_water_path_amsr.nc')
sim_codes = [f.split('/')[2].split('_')[0] for f in files]
simulations = np.array([sim for sim in simulations if sim.code in sim_codes])
model_lwp = [xr.open_dataset(file).liquid_water_path for file in files]
model_lwp = [da.where(da != fill_val) for da in model_lwp]
print(f'[INFO] loaded {len(model_lwp)} simulations')

data_mask = np.isnan(model_lwp[0][0])

## Make histograms of data

In [ ]:
mask_thresh = 1e-4  # kg m-2
# Apply domain mask and cloud threshold mask to AMSR
amsr_masked = [da.where(~data_mask).where(da>mask_thresh) for da in amsr_lwp]
amsr_valid = ma.masked_invalid([da.values for da in amsr_masked]).compressed()

# apply cloud threshold mask to model output
model_valid = [ma.masked_invalid(da.where(da>mask_thresh).values).compressed() for da in model_lwp]

# set distribution bins and call np.histogram
amsr_ndata = len(amsr_valid)
amsr_nbins = int(np.floor(np.sqrt(amsr_ndata)))
amsr_cts, amsr_bins = np.histogram(amsr_valid*1000, bins=amsr_nbins,
                                   density=True)
model_min = []
model_max = []
model_mean = []
model_median = []
model_nbins = []
model_cts = []
model_bins = []
for arr in model_valid:
    model_min.append(ma.amin(arr*1000))
    model_max.append(ma.amax(arr*1000))
    model_mean.append(ma.mean(arr*1000))
    model_median.append(ma.median(arr*1000))
    model_ndata = len(arr)
    model_nbins.append(int(np.ceil(np.sqrt(model_ndata))))
    cts, bins = np.histogram(arr*1000, bins=model_nbins[-1],
                             density=True)
    model_cts.append(cts)
    model_bins.append(bins)


## Plot figure 7

In [ ]:
print('\n\nMaking manuscript figure')
fig = plt.figure(figsize=(10*fprops.in_cm,10*fprops.in_cm), dpi=300)
amsr_min = ma.amin(amsr_valid*1000)
amsr_max = ma.amax(amsr_valid*1000)
amsr_mean = ma.mean(amsr_valid*1000)
amsr_median = ma.median(amsr_valid*1000)
print(f'AMSR min, max, mean, median = {amsr_min:.2f}, {amsr_max:.2f}, {amsr_mean:.2f}, {amsr_median:.2f}')
print(f'AMSR peak = {amsr_bins[ma.argmax(amsr_cts)]:.2f}')
# AMSR
plt.stairs(amsr_cts, amsr_bins, color='k',
           label='AMSR2', lw=1.2)
for s,sim in enumerate(simulations):
    # model LWP only, in-cloud
    plt.stairs(model_cts[s], model_bins[s],
               color=sim.colour, label=sim.label, lw=sim.linewidth)
    print(f'{sim.label} min, max, mean, median = {model_min[s]:.2f}, {model_max[s]:.2f}, {model_mean[s]:.2f}, {model_median[s]:.2f}')
    print(f'{sim.label} peak = {model_bins[s][ma.argmax(model_cts[s])]:.2f}')
plt.xlabel('LWP [g m-2]', fontsize=fprops.ax_label_fs)
plt.ylabel('PDF', fontsize=fprops.ax_label_fs)
plt.legend(fontsize=fprops.legend_fs)
plt.savefig('figures/figure7.png', bbox_inches='tight', facecolor='white')
